In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from ipywidgets import interact
from tqdm.auto import tqdm
from eventio import SimTelFile
from lstchain import mc
import glob
import math

from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
#%matplotlib widget

In [ ]:
datafile = "/Users/vdk/UnrealLST/simtel_investigations/MONOSTEREO_run1001_muon.simtel.gz"
datafile = "/Users/vdk/UnrealLST/simtel_investigations/STEREOSTEREO_run1001_muon.simtel.gz"
datafile = "/Users/vdk/UnrealLST/simtel_investigations/ARRAYSTEREO_run1001_muon.simtel.gz"
#datafile = "/home/voutsi/Work/TriggerStudies/analysis/Electrons_data/simtel_corsika_theta_52.374_az_216.698_run4.simtel.gz"
source = SimTelFile(datafile)
source.mc_run_headers[0]


In [ ]:
#FilenamesList = glob.glob('simtel_corsika_theta_66.446_az_284.017_run*.simtel.gz')
FilenamesList = glob.glob('simtel_*_run617.simtel.gz')


In [ ]:
xcores = []  
ycores = []
xcores_sim = []
ycores_sim = []
Alts = []
Azimuths = []
PE = []


In [ ]:
for filename in FilenamesList:
    print(filename)
    with SimTelFile(filename) as f:
        evt_iter = f.iter_array_events()
        xcores_temp = []
        ycores_temp = []
        for event in evt_iter:
            xcores_temp.append(event['mc_event']['xcore'])
            ycores_temp.append(event['mc_event']['ycore']) 
            #print(event['mc_event'])
        xcores.extend(xcores_temp)
        ycores.extend(ycores_temp)

In [ ]:
with SimTelFile(datafile) as f:
    evt_iter = f.iter_array_events()
    xcores_temp = []
    ycores_temp = []
    for event in evt_iter:
        xcores_temp.append(event['mc_event']['xcore'])
        ycores_temp.append(event['mc_event']['ycore']) 
        #print(event['mc_event'])
    xcores.extend(xcores_temp)
    ycores.extend(ycores_temp)

In [ ]:
# for filename in FilenamesList:
#     with SimTelFile(filename) as f:
#         evt_mc_iter = f.iter_mc_events()
#         xcores_sim_temp = []
#         ycores_sim_temp = []    
#         for event in evt_mc_iter:
#             xcores_sim_temp.append(event['mc_event']['xcore'])
#             ycores_sim_temp.append(event['mc_event']['ycore'])     
#         xcores_sim.extend(xcores_sim_temp)
#         ycores_sim.extend(ycores_sim_temp)    


with SimTelFile(datafile) as f:
    evt_mc_iter = f.iter_mc_events()
    xcores_sim_temp = []
    ycores_sim_temp = []    
    for event in evt_mc_iter:
        xcores_sim_temp.append(event['mc_event']['xcore'])
        ycores_sim_temp.append(event['mc_event']['ycore'])     
    xcores_sim.extend(xcores_sim_temp)
    ycores_sim.extend(ycores_sim_temp)

In [ ]:
print('number of simulated events', len(xcores_sim))
print('number of triggered events', len(xcores))

In [ ]:
#print("xcores", xcores)

In [ ]:
plt.hist2d(xcores_sim, ycores_sim, bins=(50, 50), cmap=plt.cm.jet)
telescope1 = [-70.93, -52.07]
telescope2 = [-35.27, 66.14]
telescope3 = [75.28, 50.49]
telescope4 = [30.91, -64.54]
# plt.scatter(telescope1[0],telescope1[1], label = 'LST1', marker = 'x', s =200, lw = 5, c = 'k')
# plt.scatter(telescope2[0],telescope2[1], label = 'LST2', marker = 'x', s =200, lw = 5, c = 'k')
# plt.scatter(telescope3[0],telescope3[1], label = 'LST3', marker = 'x', s =200, lw = 5, c = 'k')
# plt.scatter(telescope4[0],telescope4[1], label = 'LST4', marker = 'x', s =200, lw = 5, c = 'k')
plt.colorbar()
plt.ylabel('y core (m)')
plt.xlabel('x core (m)')
plt.title('Cores for simulated showers')
plt.xlim(-150,150)
plt.ylim(-150,150)
#plt.savefig('SimCoresElectrons_617.png')
plt.show()

In [ ]:
plt.hist2d(xcores, ycores, bins=(50, 50), cmap=plt.cm.jet)
plt.colorbar()
plt.ylabel('y core (m)')
plt.xlabel('x core (m)')
plt.title('Cores for showers that triggered')
plt.xlim(-150,150)
plt.ylim(-150,150)
#plt.savefig('TrigCoresElectrons_617.png')
plt.show()

In [ ]:
# Off axis angle analysis
for filename in FilenamesList:
    with SimTelFile(filename) as f:
        evt_iter = f.iter_array_events()
        Alts_temp = []
        Azimuths_temp = []
        for event in evt_iter:
            Alts_temp.append(event['mc_shower']['altitude'])
            Azimuths_temp.append(event['mc_shower']['azimuth'])  
            #print(event)
        Alts.extend(Alts_temp)
        Azimuths.extend(Azimuths_temp)
print("Alts", Alts, "length", len(Alts))

In [ ]:
theta1 = (math.pi/2) - source.mc_run_headers[0]['alt_range'][1]
phi1 = source.mc_run_headers[0]['az_range'][1]
print("theta1",theta1,"phi1",phi1)

OffAxis = []

Dir1 = [math.sin(theta1)*math.cos(phi1), math.sin(theta1)*math.sin(phi1), math.cos(theta1)]
print(Dir1)
for i,j in zip(Alts, Azimuths):
    Zd = (math.pi/2) - i
    Az = j
    Dir2 = [math.sin(Zd)*math.cos(Az), math.sin(Zd)*math.sin(Az), math.cos(Zd)]
    DirVec1 = np.array(Dir1) 
    DirVec2 = np.array(Dir2)
    alpha = math.acos(np.dot(DirVec1,DirVec2))
    OffAxis.append((alpha*180)/math.pi)

In [ ]:
#bins = range(0,10,1)
bins = np.linspace(0,10,20)
fig, axes = plt.subplots()
bins_content, bins_edge, patches = plt.hist(OffAxis, bins = bins, label='Protons, DT=300mV, viewcone = 10')
plt.title('Electrons, viewcone = 8')
plt.xlabel('Off axis angle (deg)')
plt.show()
#plt.savefig('Electrons_viewcone.png')

In [ ]:
FilenamesList = glob.glob('simtel_*_run617.simtel.gz')
xcores = []  
ycores = []
xcores_sim = []
ycores_sim = []
Alts = []
Azimuths = []
PE = []
for filename in FilenamesList:
    print(filename)
    with SimTelFile(filename) as f:
        evt_iter = f.iter_array_events()
        xcores_temp = []
        ycores_temp = []
        for event in evt_iter:
            xcores_temp.append(event['mc_event']['xcore'])
            ycores_temp.append(event['mc_event']['ycore']) 
            #print(event['mc_event'])
        xcores.extend(xcores_temp)
        ycores.extend(ycores_temp)

for filename in FilenamesList:
    with SimTelFile(filename) as f:
        evt_mc_iter = f.iter_mc_events()
        xcores_sim_temp = []
        ycores_sim_temp = []    
        for event in evt_mc_iter:
            xcores_sim_temp.append(event['mc_event']['xcore'])
            ycores_sim_temp.append(event['mc_event']['ycore'])     
        xcores_sim.extend(xcores_sim_temp)
        ycores_sim.extend(ycores_sim_temp)

plt.hist2d(xcores_sim, ycores_sim, bins=(50, 50), cmap=plt.cm.jet)
plt.colorbar()
plt.ylabel('y core (m)')
plt.xlabel('x core (m)')
plt.title('Cores for simulated showers')
plt.xlim(-150,150)
plt.ylim(-150,150)
#plt.savefig('SimCoresElectrons_617.png')


plt.hist2d(xcores, ycores, bins=(50, 50), cmap=plt.cm.jet)
plt.colorbar()
plt.ylabel('y core (m)')
plt.xlabel('x core (m)')
plt.title('Cores for showers that triggered')
plt.xlim(-150,150)
plt.ylim(-150,150)
#plt.savefig('TrigCoresElectrons_617.png')